In [30]:
# read text file "Nebraska.population" and store it as a list called population
population = list()

#open text file to be read
filepath = r"C:\Users\Keegan\Desktop\\" 
filename = "Nebraska.population.txt"
file = open( filepath + filename , "r")


#read in a new county population
line = file.readline()

while line != "":
    #split line into two "words":
    # word[0]: the county's number
    # word[1]: the county's population
    words = line.split()
    county_number = words[0]
    county_population = int(words[1])
    
    # append to population list
    population.append(county_population)
    
    #read next line
    line=file.readline()

file.close()
print("population = ", population)


population =  [6129, 9998, 11308, 818, 7547, 15740, 36288, 736, 11055, 614, 36691, 3966, 158840, 36970, 20234, 2538, 3812, 517110, 25241, 5713, 5505, 5217, 9182, 2756, 4500, 6489, 6000, 6274, 824, 763, 8852, 632, 5406, 478, 3821, 2044, 3423, 9188, 9124, 8395, 10435, 32237, 5890, 10939, 46102, 6858, 24326, 20780, 647, 6685, 2049, 1311, 1941, 2773, 2099, 5228, 2908, 690, 4959, 6542, 7845, 3145, 10515, 21006, 460, 5469, 16750, 9595, 8701, 7248, 5042, 58607, 4260, 34876, 3225, 3735, 967, 22311, 13665, 539, 2970, 1526, 285407, 31364, 9139, 2057, 8368, 6940, 7266, 8363, 14200, 3152, 2008]


In [31]:
# we are to solve the following task:
# input: a population vector, desired number of districts k
# output: a partition of the populations into k districts (not necessarily connected!) 
#            to minimize the difference between most and least populated districts

import gurobipy as gp
from gurobipy import GRB

In [49]:
c = len(population) # this is the number of counties
d = 3                  #this is the number of districts for Nebraska

#create model
m = gp.Model()

x= m.addVars(c,k,vtype=GRB.BINARY) # Either county i is in district j or it is not "0"
y = m.addVar()                     #population of the least populated district
z= m.addVar()                      #population of the most populated district

In [52]:
m.setObjective(z-y,GRB.MINIMIZE)   # the objective is to minimize the population deviation between the districts

m.addConstrs(sum(x[i,j] for j in range(d)) == 1 for i in range(c)) #Each county can only be in one district

m.addConstrs(y<= sum( population[i] * x[i,j] for i in range(c)) for j in range(d)) #each district must have at least population of y

m.addConstrs( sum( population[i] * x[i,j] for i in range(c))  <= z for j in range(d)) # each district can have at most the population of z
m.update()

In [54]:
m.optimize() 
#solve the IP model note: this model does not impose contiguity or other applicable Nebraska Law


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 198 rows, 281 columns and 1686 nonzeros
Model fingerprint: 0xb780db20
Variable types: 2 continuous, 279 integer (279 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 1 (0.00s)
Loaded MIP start from previous solve with objective 1

Presolve removed 99 rows and 0 columns
Presolve time: 0.00s
Presolved: 99 rows, 281 columns, 843 nonzeros
Variable types: 0 continuous, 281 integer (279 binary)

Root relaxation: objective -1.164153e-10, 91 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.00000    0    6    1.00000   

In [56]:
print ("The minimum population deviation for each district is",m.objval,"people.")
#print the population deviation

# pull the districts and their populations
district = [ [i for i in range(c) if x[i,j].x >0.5] for j in range(d)]
district_population = [ sum(population[i] for i in district) for district in district]

#print the information
for j in range(d):
    print("District", j,"had population of",district_population[j],"and contains the following counties",districts[j])
    

The minimum population deviation for each district is 1.0 people.
District 0 had population of 608780 and contains the following counties [0, 3, 7, 10, 11, 12, 13, 16, 18, 19, 23, 24, 25, 26, 27, 29, 30, 32, 34, 35, 36, 37, 38, 41, 42, 45, 46, 47, 49, 50, 51, 53, 54, 57, 59, 60, 63, 64, 67, 69, 71, 72, 74, 78, 79, 80, 84, 85, 86]
District 1 had population of 608781 and contains the following counties [4, 8, 17, 20, 21, 48, 55, 70, 76, 77, 81, 88, 90, 91, 92]
District 2 had population of 608780 and contains the following counties [1, 2, 5, 6, 9, 14, 15, 22, 28, 31, 33, 39, 40, 43, 44, 52, 56, 58, 61, 62, 65, 66, 68, 73, 75, 82, 83, 87, 89]
